# running automatic fitting in ROD

before running the cells below make sure you have done the following:
- have a version of RODautofitting.py in the same directory as where you have saved this notebook
- made a copy of rod_init.mac in the ROD directory saved as rod_init_template.mac
- saved a copy of fitlogloop5.mac and fitlogloop5run.mac in the ROD directory. Use the text below, replacing 'path\to\fitting\folder'  with you own chosen folder. This will save the output of 5 fitting runs to fitting_#.par files

fitlogloop5.mac - carries out 5 ASA fitting loops and saves output
```
li par "path\to\fitting\folder\fitting_0.par" start_parameters
fit control open "path\to\fitting\folder\\fitrun.log" return return
fit asa pl allbo
li par "path\to\fitting\folder\\fitting_1.par" results_from_loop_1
fit asa pl allbo
li par "path\to\fitting\folder\\fitting_2.par" results_from_loop_2
fit asa pl allbo
li par "path\to\fitting\folder\\fitting_3.par" results_from_loop_3
fit asa pl allbo
li par "path\to\fitting\folder\\fitting_4.par" results_from_loop_4
fit asa pl allbo
li par "path\to\fitting\folder\\fitting_5.par" results_from_loop_5
fit control close return return
mac modelout
```

fitlogloop5run.mac - carries out 5 run fitting loops and saves output
```
li par "path\to\fitting\folder\fitting_0.par" start_parameters
fit control open "path\to\fitting\folder\\fitrun.log" return return
fit run pl allbo
li par "path\to\fitting\folder\\fitting_1.par" results_from_loop_1
fit run pl allbo
li par "path\to\fitting\folder\\fitting_2.par" results_from_loop_2
fit run pl allbo
li par "path\to\fitting\folder\\fitting_3.par" results_from_loop_3
fit run pl allbo
li par "path\to\fitting\folder\\fitting_4.par" results_from_loop_4
fit run pl allbo
li par "path\to\fitting\folder\\fitting_5.par" results_from_loop_5
fit control close return return
mac modelout
```

## Define all file paths
run cell below to load in the path information for where to find and/or save all the important files

In [ ]:
from RODautofitting import * 
import os

# give the path to where the final 5th fitting run is saved for each loop of 5
fit5path=fr"\path\to\fitting\folder\fitting_5.par"
folderpath='define path here'
# define the files to load into ROD using a dictionary
files={}
files['fit']= fr"{folderpath}\path\to\fit\file"
files['dat']= fr"{folderpath}path\to\dat\file"
files['bul']= fr"{folderpath}\path\to\bulk\file"
files['loadmac']= fr"{folderpath}\path\to\macro\file"  # e.g. for loading matrices
files['par']= fr"{folderpath}\path\to\par\file"  #used as starting values
files['fitmac']=fr"{folderpath}\path\to\fittingmacro\file"

# create model name based on fitfile
modname = files['fit'].split(r"\\")[-1]

# set computer specific location for rod program and its init.mac file
rodexe=fr"\path\to\anarod_standard_1-7_mingw\rod.exe"
initfile=fr"\path\to\\anarod_standard_1-7_mingw\rod_init.mac"

#make a copy of initial rod_init.mac before any edits saved as rod_init_template.mac
templateinitfile=fr"\path\to\anarod_standard_1-7_mingw\rod_init_template.mac"

# set where to save dummy file used to flag when fitting loop is done
donefile=fr"\path\to\to\done\file"

# set path where fitting results are saved as defined in macro fitlogloop5.mac
fit5path=fr"\path\to\path\to\fitting\folder\fitting_5.par"


## start the fitting routine
Running the cell below starts the fitting cycles which does a series of ASA fits and then run fits until the chi squared values have stabilised

In [ ]:
#create a ASA loop macro based on files dictionary
createmacro(modname,files,files['par'],donefile,'ASA')

#do an initial ASA fitting round
domacrofits(donefile,rodexe)

#define where the outputs are being save for each loop macro
run = os.path.dirname(fit5path)+rf"\fitting"

#calculate chi-squared information for first fit
fitlist, fitted, ranges, fitdf=calcfitlist(run, 5)
asacount = 1
print(f"asa loop {asacount}")

#enter a loop that continues running rounds of 5 ASA fits until the final three chi squared values are within 0.01 of each other
endasachi,asacount=fitloop(
    modname, fitlist, files, fit5path, donefile, rodexe, asacount, run,'ASA'
    )

# once the ASA fitting gave a stable answer copy the parameters to  a set location
shutil.copy(
    f"{fit5path}",fr"{folderpath}\path\to\saving\location\asafit_{modname.replace('.fit','')}.par"
    )

# set starting parameters to the last of the ASA fits
startvals = fit5path

# create a run loop macro
createmacro(modname, files, startvals, donefile,'RUN')

# do an initial run fitting round
domacrofits(donefile, rodexe)

#calculate chi-squared information for first fit
fitlist, fitted, ranges, fitdf = calcfitlist(run,5)
runcount = 1
print(f'run loop {runcount}')

#enter a loop that continues running rounds of 5 run fits until the final three chi squared values are within 0.005 of each other
endrunchi,runcount=fitloop(
    modname,fitlist,files,fit5path,donefile,rodexe,runcount,run,'RUN'
    )

#once the run fitting gave a stable answer copy the parameters to  a set location
shutil.copy(f"{fit5path}",fr"{folderpath}\path\to\saving\location\runfit_{modname.replace('.fit','')}.par")